#Temporal Planning using the Unified Planning framework

This python notebook shows how to use the unified planning library to model temporal problems.

## Setup the library

We install (from github) the unified planning library.

In [ ]:
# begin of installation

In [ ]:
!pip install --pre -U unified-planning

Then, we download and install tamer

In [ ]:
!rm -rf up-tamer && git clone https://github.com/aiplan4eu/up-tamer && pip install up-tamer/

We are now ready to use the Unified-Planning library!

In [ ]:
# end of installation

## Demo


In this demo we show how to model a temporal planning problem using the Unified Planning library.

Temporal planning extends classical planning introducing durative actions, timed effects and timed goals.



We start importing the shortcuts.

In [ ]:
from unified_planning.shortcuts import *

Now we start to model the MatchCellar problem.



### Creating the problem

#### Classical part

First, we define the `UserTypes` and the `Fluents`.

In [ ]:
Match = UserType('Match')
Fuse = UserType('Fuse')

handfree = Fluent('handfree')
light = Fluent('light')
match_used = Fluent('match_used', BoolType(), m=Match)
fuse_mended = Fluent('fuse_mended', BoolType(), f=Fuse)

We create a `Problem`, we add the fluents to it and we set their initial values.

In [ ]:
problem = Problem('MatchCellar')

problem.add_fluent(handfree)
problem.add_fluent(light)
problem.add_fluent(match_used, default_initial_value=False)
problem.add_fluent(fuse_mended, default_initial_value=False)

problem.set_initial_value(light, False)
problem.set_initial_value(handfree, True)

We create the objects and we add them to the problem.

In [ ]:
fuses = [Object(f'f{i}', Fuse) for i in range(3)]
matches = [Object(f'm{i}', Match) for i in range(3)]

problem.add_objects(fuses)
problem.add_objects(matches)

#### Temporal part

Now we start with the temporal aspects creating two durative actions.

A durative action has a duration, a set of conditions associated to an interval/timing and a set of effects associated to a timing.

We define the `light_match` action setting a fixed duration and defining a condition at its start and three effects at its end.

In [ ]:
light_match = DurativeAction('light_match', m=Match)
m = light_match.parameter('m')
light_match.set_fixed_duration(6)
light_match.add_condition(StartTiming(), Not(match_used(m)))
light_match.add_effect(StartTiming(), match_used(m), True)
light_match.add_effect(StartTiming(), light, True)
light_match.add_effect(EndTiming(), light, False)
problem.add_action(light_match)
print(light_match)

Defining the `mend_fuse` action we defined also a condition over an interval.

In [ ]:
mend_fuse = DurativeAction('mend_fuse', f=Fuse)
f = mend_fuse.parameter('f')
mend_fuse.set_fixed_duration(5)
mend_fuse.add_condition(StartTiming(), handfree)
mend_fuse.add_condition(ClosedTimeInterval(StartTiming(), EndTiming()), light)
mend_fuse.add_effect(StartTiming(), handfree, False)
mend_fuse.add_effect(EndTiming(), fuse_mended(f), True)
mend_fuse.add_effect(EndTiming(), handfree, True)
problem.add_action(mend_fuse)
print(mend_fuse)

We conclude the modeling defining three goals at the end of the execution.

In [ ]:
for f in fuses:
  problem.add_timed_goal(EndTiming(), fuse_mended(f))

print(problem)

### Solving the problem

The unified_planning can automatically select, among the available planners installed on the system, one that is able to handle the temporal features of the problem.

In [ ]:
with OneshotPlanner(problem_kind=problem.kind) as planner:
    result = planner.solve(problem)
    plan = result.plan
    if plan is not None:
        print("%s returned:" % planner.name)
        for start, action, duration in plan.timed_actions:
            print("%s: %s [%s]" % (float(start), action, float(duration)))
    else:
        print("No plan found.")